In [ ]:
from __future__ import print_function
import torch
import torch.utils.data
from torch import nn, optim
from torch.autograd import Variable
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image


"""parser = argparse.ArgumentParser(description='VAE MNIST Example')
parser.add_argument('--batch-size', type=int, default=128, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='enables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')"""


train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, #download=True,
                   transform=transforms.ToTensor()),
    batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=128, shuffle=True)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784+10, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()


def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784))

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # Normalise by same number of elements as in reconstruction
    KLD /= 128 * 784

    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-3)


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    for i, (data, _) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        test_loss += loss_function(recon_batch, data, mu, logvar).data[0]
        if i == 0:
            n = min(data.size(0), 8)
            comparison = torch.cat([data[:n],
                                  recon_batch.view(128, 1, 28, 28)[:n]])
            save_image(comparison.data.cpu(),
                     'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


for epoch in range(1, 10 + 1):
    train(epoch)
    test(epoch)
    sample = Variable(torch.randn(64, 20))
    sample = model.decode(sample).cpu()
    save_image(sample.data.view(64, 1, 28, 28),
               'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.005491
Train Epoch: 1 [1280/60000 (2%)]	Loss: 0.002928
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.002330
Train Epoch: 1 [3840/60000 (6%)]	Loss: 0.002275
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.002189
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.002160
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.002112
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.002047
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.001929
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.001878
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.001832
Train Epoch: 1 [14080/60000 (23%)]	Loss: 0.001723
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.001741
Train Epoch: 1 [16640/60000 (28%)]	Loss: 0.001646
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.001626
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.001591
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.001648
Train Epoch: 1 [21760/60000 (36%)]	Loss: 0.001567
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.001576
Train Epoch: 1 [24320/60000 (41%)]	Loss: 0.001554
Train Epoch: 1 [

Train Epoch: 4 [26880/60000 (45%)]	Loss: 0.001076
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.001094
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.001087
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.001144
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.001084
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.001117
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.001116
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.001140
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.001092
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.001118
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.001101
Train Epoch: 4 [40960/60000 (68%)]	Loss: 0.001120
Train Epoch: 4 [42240/60000 (70%)]	Loss: 0.001101
Train Epoch: 4 [43520/60000 (72%)]	Loss: 0.001125
Train Epoch: 4 [44800/60000 (75%)]	Loss: 0.001082
Train Epoch: 4 [46080/60000 (77%)]	Loss: 0.001098
Train Epoch: 4 [47360/60000 (79%)]	Loss: 0.001116
Train Epoch: 4 [48640/60000 (81%)]	Loss: 0.001087
Train Epoch: 4 [49920/60000 (83%)]	Loss: 0.001116
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.001133


Train Epoch: 7 [52480/60000 (87%)]	Loss: 0.001089
Train Epoch: 7 [53760/60000 (90%)]	Loss: 0.001121
Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.001018
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.001094
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.001118
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.001085
====> Epoch: 7 Average loss: 0.0011
====> Test set loss: 0.0010
Train Epoch: 8 [0/60000 (0%)]	Loss: 0.001057
Train Epoch: 8 [1280/60000 (2%)]	Loss: 0.001062
Train Epoch: 8 [2560/60000 (4%)]	Loss: 0.001056
Train Epoch: 8 [3840/60000 (6%)]	Loss: 0.001111
Train Epoch: 8 [5120/60000 (9%)]	Loss: 0.001093
Train Epoch: 8 [6400/60000 (11%)]	Loss: 0.001069
Train Epoch: 8 [7680/60000 (13%)]	Loss: 0.001081
Train Epoch: 8 [8960/60000 (15%)]	Loss: 0.001083
Train Epoch: 8 [10240/60000 (17%)]	Loss: 0.001070
Train Epoch: 8 [11520/60000 (19%)]	Loss: 0.001064
Train Epoch: 8 [12800/60000 (21%)]	Loss: 0.001117
Train Epoch: 8 [14080/60000 (23%)]	Loss: 0.001093
Train Epoch: 8 [15360/60000 (26%)]	Loss: 0.001044
Tr